In [1]:
# Import libraries
import pandas as pd
import numpy as np

In [2]:
# Import dataset
df = pd.read_csv('res/TMDB_all_movies_full.csv')

df

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,budget,imdb_id,...,spoken_languages,cast,director,director_of_photography,writers,producers,music_composer,imdb_rating,imdb_votes,poster_path
0,2,Ariel,7.100,346.0,Released,1988-10-21,0.0,73.0,0.0,tt0094675,...,suomi,"Merja Pulkkinen, Eetu Hilkamo, Turo Pajala, Es...",Aki Kaurismäki,Timo Salminen,Aki Kaurismäki,Aki Kaurismäki,NaN,7.4,9224.0,/ojDg0PGvs6R9xYFodRct2kdI6wC.jpg
1,3,Shadows in Paradise,7.293,409.0,Released,1986-10-17,0.0,74.0,0.0,tt0092149,...,"suomi, English, svenska","Esko Nikkari, Mari Rantasila, Marina Martinoff...",Aki Kaurismäki,Timo Salminen,Aki Kaurismäki,Mika Kaurismäki,NaN,7.4,8053.0,/nj01hspawPof0mJmlgfjuLyJuRN.jpg
2,5,Four Rooms,5.863,2696.0,Released,1995-12-09,4257354.0,98.0,4000000.0,tt0113101,...,English,"Alicia Witt, Ione Skye, Sammi Davis, Lana McKi...","Allison Anders, Alexandre Rockwell, Robert Rod...","Andrzej Sekula, Phil Parmet, Rodrigo García, G...","Allison Anders, Alexandre Rockwell, Robert Rod...","Lawrence Bender, Alexandre Rockwell, Quentin T...",Combustible Edison,6.7,114367.0,/75aHn1NOYXh4M7L5shoeQ6NGykP.jpg
3,6,Judgment Night,6.500,351.0,Released,1993-10-15,12136938.0,109.0,21000000.0,tt0107286,...,English,"Deirdre Kelly, Peter Greene, Will Zahrn, Jerem...",Stephen Hopkins,Peter Levy,"Jere Cunningham, Lewis Colick","Marilyn Vance, Lloyd Segan, Gene Levy",Alan Silvestri,6.6,20035.0,/3rvvpS9YPM5HB2f4HYiNiJVtdam.jpg
4,8,Life in Loops (A Megacities RMX),7.500,27.0,Released,2006-01-01,0.0,80.0,42000.0,tt0825671,...,"English, हिन्दी, 日本語, Pусский, Español",NaN,Timo Novotny,Wolfgang Thaler,"Michael Glawogger, Timo Novotny","Ulrich Gehmacher, Timo Novotny",NaN,8.1,285.0,/7ln81BRnPR2wqxuITZxEciCe1lc.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1108738,1496305,The Seagull,0.000,0.0,Released,NaN,0.0,0.0,0.0,tt32378695,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1108739,1496306,赖纳·维尔纳·法斯宾德作品 1966-1969,0.000,0.0,Released,NaN,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1108740,1496308,Pandora's Paradox 2: The Musical,0.000,0.0,Released,NaN,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1108741,2662126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"Johannes Allmayer, Jules Waringo, Vicky Krieps...",Florian Gallenberger,Kolja Brandt,"Florian Gallenberger, Torsten Wenzel, Tom Kinn...","Nicolas Steil, Benjamin Herrmann",NaN,NaN,NaN,NaN


In [3]:
# Drop irrelevant columns
df = df.drop(columns=["poster_path", "overview", "tagline", "original_title"])

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1108743 entries, 0 to 1108742
Data columns (total 24 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   id                       1108743 non-null  int64  
 1   title                    1108723 non-null  object 
 2   vote_average             1108741 non-null  float64
 3   vote_count               1108741 non-null  float64
 4   status                   1108741 non-null  object 
 5   release_date             979517 non-null   object 
 6   revenue                  1108741 non-null  float64
 7   runtime                  1108741 non-null  float64
 8   budget                   1108741 non-null  float64
 9   imdb_id                  623121 non-null   object 
 10  original_language        1108741 non-null  object 
 11  popularity               1108741 non-null  float64
 12  genres                   791254 non-null   object 
 13  production_companies     516184 non-null  

In [5]:
# Check if id viable to be used as index
df.id.is_unique

True

In [6]:
# Returned true, set id column as index
df = df.set_index("id")

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1108743 entries, 2 to 5180730
Data columns (total 23 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   title                    1108723 non-null  object 
 1   vote_average             1108741 non-null  float64
 2   vote_count               1108741 non-null  float64
 3   status                   1108741 non-null  object 
 4   release_date             979517 non-null   object 
 5   revenue                  1108741 non-null  float64
 6   runtime                  1108741 non-null  float64
 7   budget                   1108741 non-null  float64
 8   imdb_id                  623121 non-null   object 
 9   original_language        1108741 non-null  object 
 10  popularity               1108741 non-null  float64
 11  genres                   791254 non-null   object 
 12  production_companies     516184 non-null   object 
 13  production_countries     668768 non-null   obje

In [8]:
# Remove movies with no title
df = df.dropna(subset=["title"])

### Based on the TMDB content score, a movie with complete information should have

- a director
- a writer
- four or more cast members
- an overview
- a poster
- a backdrop
- a release date
- a trailer


Source: https://www.themoviedb.org/faq/website

In [9]:
df = df.dropna(subset=["director", "release_date", "genres", "vote_average"])

In [10]:
# Remove movies (not documentaries) with no cast
df = df[(df["genres"].isin(["Documentary"]) | df["cast"].notna())]

In [11]:
# Most proper movies have four actors, regex for minimum four items, delimited by comma
minimum_four_items = r"(?:[^,]*,){3,}"

# Filter for movies (not documentaries) with at least four cast members
df = df[(df["genres"].isin(["Documentary"]) | df.cast.str.contains(minimum_four_items))]

In [12]:
df.isnull().sum()

title                           0
vote_average                    0
vote_count                      0
status                          0
release_date                    0
revenue                         0
runtime                         0
budget                          0
imdb_id                    120669
original_language               0
popularity                      0
genres                          0
production_companies       166552
production_countries        93639
spoken_languages            98354
cast                        66251
director                        0
director_of_photography    304737
writers                    143608
producers                  260567
music_composer             417579
imdb_rating                194192
imdb_votes                 194192
dtype: int64

In [13]:
pd.set_option('display.max_columns', None)

In [14]:
pd.set_option("display.max_rows", 100)

In [15]:
df.sort_values(by="revenue", ascending=False).head()

,title,vote_average,vote_count,status,release_date,revenue,runtime,budget,imdb_id,original_language,popularity,genres,production_companies,production_countries,spoken_languages,cast,director,director_of_photography,writers,producers,music_composer,imdb_rating,imdb_votes
id,,,,,,,,,,,,,,,,,,,,,,,
1270893,TikTok Rizz Party,10.000,1.0,Released,2024-04-01,3.000000e+09,180.0,250000000.0,NaN,en,0.0000,"Drama, Comedy",Warner Bros.,United States of America,NaN,"Matt Damon, Ben Affleck, Quandale Dingle, Tnic...",Martin Scorsese,NaN,Martin Scorsese,"Martin Scorsese, Steven Spielberg, Christopher...",NaN,NaN,NaN
19995,Avatar,7.590,32295.0,Released,2009-12-15,2.923706e+09,162.0,237000000.0,tt0499549,en,22.6517,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Español","Larry Rew, Lisa Roumain, Jon Curry, Jahnel Cur...",James Cameron,Mauro Fiore,James Cameron,"Jon Landau, Laeta Kalogridis, Colin Wilson, Ja...",James Horner,7.9,1433482.0
299534,Avengers: Endgame,8.239,26407.0,Released,2019-04-24,2.799439e+09,181.0,356000000.0,tt4154796,en,62.6115,"Adventure, Science Fiction, Action",Marvel Studios,United States of America,"English, 日本語,","Anthony Mackie, James D'Arcy, Aaron Lazar, Tom...","Anthony Russo, Joe Russo",Trent Opaloch,"Joe Simon, Bill Mantlo, Jim Starlin, Don Heck,...","Jon Favreau, Trinh Tran, Michael Grillo, Stan ...",Alan Silvestri,8.4,1362816.0
76600,Avatar: The Way of Water,7.600,12602.0,Released,2022-12-14,2.320250e+09,192.0,460000000.0,tt1630029,en,23.2497,"Science Fiction, Adventure, Action","20th Century Studios, Lightstorm Entertainment",United States of America,English,"Benjamin Hoetjes, Bailey Bass, Isaac Te Reina,...",James Cameron,Russell Carpenter,"Shane Salerno, Amanda Silver, James Cameron, J...","Richard Baneham, David Valdes, James Cameron, ...",Simon Franglen,7.5,536313.0
597,Titanic,7.905,26040.0,Released,1997-11-18,2.264162e+09,194.0,200000000.0,tt0120338,en,32.7318,"Drama, Romance","Paramount Pictures, 20th Century Fox, Lightsto...",United States of America,"English, Français, Deutsch, svenska, Italiano,...","Anatoly M. Sagalevitch, Michael Ensign, Victor...",James Cameron,Russell Carpenter,James Cameron,"James Cameron, Jon Landau, Rae Sanchini",James Horner,7.9,1342640.0


In [16]:
# Remove suspicious "movies" with high revenue but few ratings
df = df.drop(df[(df["revenue"] > 500000000) & (df["vote_count"] < 50)].index)


In [17]:
# The chances of real 10M+ USD movies having no IMDB page is pretty small 
possible_fake_values = df[(df["imdb_id"].isna()) & ((df["revenue"] > 10000000) | (df["budget"] > 10000000))]

df = df.drop(possible_fake_values.index)

In [18]:
df.shape[0]

517342

In [19]:
hours = 60

long_movies = df[df["runtime"] > 6 * hours]

long_movies_no_docus = long_movies[~long_movies["genres"].str.contains("Docu")]

long_movies_no_docus.tail(10)

,title,vote_average,vote_count,status,release_date,revenue,runtime,budget,imdb_id,original_language,popularity,genres,production_companies,production_countries,spoken_languages,cast,director,director_of_photography,writers,producers,music_composer,imdb_rating,imdb_votes
id,,,,,,,,,,,,,,,,,,,,,,,
1484788,Marvel's Daredevil,0.0,0.0,Released,2015-04-10,290000000.0,780.0,56000000.0,tt3322312,en,0.1071,"Crime, Drama, Action, Adventure",NaN,NaN,English,"Vondie Curtis-Hall, Jay Ali, Wilson Bethel, Jo...","Stephen Surjik, Phil Abraham",NaN,Drew Goddard,Drew Goddard,NaN,8.6,507466.0
1485290,Marvel's Jessica Jones,0.0,0.0,Released,2018-11-20,0.0,640.0,0.0,tt2357547,en,0.0643,"Drama, Science Fiction, Fantasy, Crime",NaN,NaN,NaN,"Mike Colter, Rachael Taylor, John Ventimiglia,...","Jennifer Getzinger, Stephen Surjik",NaN,Melissa Rosenberg,"Melissa Rosenberg, Scott Reynolds",NaN,7.8,234030.0
1485430,The Days and Years of Nikolai Batygin,0.0,0.0,Released,1987-11-01,0.0,371.0,0.0,tt0092341,ru,0.0286,"Drama, TV Movie",Ekran,Soviet Union,NaN,"Valentin Pechnikov, Vadim Spiridonov, Mariya Z...",Leonid Pchyolkin,Vladimir Trofimov,Anatoli Galiyev,NaN,NaN,7.0,11.0
1485933,Marvel's Agent Carter,0.0,0.0,Released,2015-01-06,0.0,756.0,0.0,tt3475734,en,0.0000,"Adventure, Action, Science Fiction, Drama",NaN,United States of America,NaN,"Shea Whigham, Chad Michael Murray, Enver Gjoka...","Louis D'Esposito, Joe Russo",NaN,"Christopher Markus, Stephen McFeely","Christopher Markus, Stephen McFeely",NaN,7.8,92173.0
1487376,Sirens,0.0,0.0,Released,2025-05-22,0.0,450.0,0.0,NaN,en,0.0286,"Drama, Comedy",NaN,NaN,English,"Julianne Moore, Bill Camp, Glenn Howerton, Mil...",Molly Smith Metzler,NaN,NaN,"Dani Gorin, Molly Smith Metzler, Nicole Kassel...",NaN,NaN,NaN
1487613,Split,0.0,0.0,Released,2004-03-07,0.0,700.0,0.0,tt0131597,en,0.0643,"Science Fiction, Action, Crime, Thriller",NaN,United States of America,"普通话, English, 日本語, Pусский","Chrismandu, Sean Miller, Debbie Rochon, Rodney...",Chaim Bianco,NaN,Chaim Bianco,Chaim Bianco,NaN,5.1,34.0
1488143,Even Though We're Adults,10.0,1.0,Released,2025-04-26,0.0,383.0,0.0,NaN,ja,0.1389,"Drama, Romance",Hulu Japan,NaN,日本語,"Mayuko Kawakita, Renn Kiriyama, Shogo Hama, Yu...",Atsushi Kaneshige,NaN,Takako Shimura,"Masahiko Shibuya, Mari Iwanaga, Masataka Katsu...",NaN,NaN,NaN
1488912,Arcane,0.0,0.0,Released,2021-11-06,108000000.0,720.0,250000000.0,tt11126994,en,0.2868,"Animation, Action, Adventure, Science Fiction,...",NaN,NaN,NaN,"Katie Leung, Kevin Alejandro, Ella Purnell, Mi...","Arnaud Delord, Pascal Charrue",NaN,"Christian Linke, Alex Yee, Conor Sheehy, Avigh...",NaN,NaN,9.0,398443.0
1492359,WRMS LORE,0.0,0.0,Released,2024-10-08,0.0,999.0,0.0,NaN,en,0.0000,"Action, Adventure, Comedy, Crime, Drama, Fanta...",WRMS Productions,NaN,English,"Ethan Flores, Jonah Hill, Abby Cates, Joseph C...",Avery Hand,NaN,"Avery Hands, Ethan Flore, Carter Martin",Carter Martin,NaN,NaN,NaN


In [20]:
# Lot of these "movies" are people who have added their fav TV-series, let's remove
df = df.drop(long_movies_no_docus.index)

In [21]:
df.groupby("status").size()

status
Canceled                5
In Production         781
Planned                89
Post Production       733
Released           515388
Rumored                 6
dtype: int64

In [22]:
# Remove movies not released or in post production
df = df[(df["status"].isin(["Released"]) | df["status"].isin(["Post Production"]))]

In [23]:
df_has_imdb = df["imdb_id"].notna()

# Removing duplicates based on IMDBid
df = df[(~df.duplicated(subset="imdb_id") | (df["imdb_id"].isna()))]

In [24]:
df.tail(25)

,title,vote_average,vote_count,status,release_date,revenue,runtime,budget,imdb_id,original_language,popularity,genres,production_companies,production_countries,spoken_languages,cast,director,director_of_photography,writers,producers,music_composer,imdb_rating,imdb_votes
id,,,,,,,,,,,,,,,,,,,,,,,
1496057,Kasımpaşalı Recep,0.0,0.0,Released,1965-01-01,0.0,0.0,0.0,tt0285648,tr,0.0143,Adventure,"Televizyon Film, Kılıç Film",NaN,NaN,"Yılmaz Güney, Emin Demirtay, Muharrem Gürses, ...",Nuri Akıncı,NaN,"Yılmaz Güney, Yaşar Akıncı",Nevzat Kığı,NaN,5.5,57.0
1496077,Five Nights at Freddy's S.L. - The Movie,8.0,1.0,Released,2018-04-20,0.0,1.0,0.0,NaN,en,0.0857,"Horror, Animation",NaN,NaN,NaN,"Kelvin, Steve, Leanne Osmond, Wade Dauksch, Ba...",Secret4Studio,NaN,NaN,Secret4Studio,NaN,NaN,NaN
1496083,Basil King Presents: Bobby's Sick Day,0.0,0.0,Released,2025-05-27,0.0,9.0,1000.0,NaN,en,0.0071,Comedy,Static Heart Productions,NaN,NaN,"Michael Fischer, Shaun Meglen, Taylor Fredrick...",William F. Reed,Jim McKay,"Ryan Anthony Castro, Isaac Scego, Taylor Fredr...","Ryan Anthony Castro, Sheilava, Taylor Fredrick...",NaN,NaN,NaN
1496093,On Sonder Hill,0.0,0.0,Released,2025-06-11,0.0,0.0,0.0,NaN,en,0.0071,Drama,NaN,NaN,NaN,"David Mason, Carol Fletcher, James Sprague, Ma...",Jacob Perkins,Harry Holmes,Holly Houston,Ben Mirfin,NaN,NaN,NaN
1496095,Mädchen weinen nicht,0.0,0.0,Released,2025-03-17,0.0,0.0,0.0,NaN,de,0.0143,Documentary,NaN,Germany,Deutsch,NaN,Sigrid Klausmann,NaN,NaN,NaN,NaN,NaN,NaN
1496098,Thomas Hitzlsperger und die Township Kinder,0.0,0.0,Released,2010-06-28,0.0,0.0,0.0,NaN,de,0.0071,Documentary,NaN,Germany,Deutsch,Thomas Hitzlsperger,Sigrid Klausmann,NaN,NaN,NaN,NaN,NaN,NaN
1496110,Die Kunst der Provokation,0.0,0.0,Released,2017-08-23,0.0,0.0,0.0,NaN,de,0.0071,Documentary,NaN,Germany,Deutsch,NaN,Thomas Lauterbach,NaN,NaN,NaN,NaN,NaN,NaN
1496115,Paranoid Android,0.0,0.0,Released,2025-04-25,0.0,5.0,1500.0,NaN,en,0.0071,"Science Fiction, Thriller",Rough Cut Collective,NaN,English,"Kylie Stewart, JeanDaniel Kaba, Daniel Gorbett...",Max Ries,Eliana D'Astici,Max Ries,Gabriel Story,NaN,NaN,NaN
1496119,"Wer sucht, erfindet - Vom Erfolg eines Familie...",0.0,0.0,Released,2002-11-21,0.0,0.0,0.0,NaN,de,0.0071,Documentary,NaN,Germany,Deutsch,NaN,Sabine Willmann,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
new_entries = df[df.index > 1495000]

# Many new entries have incomplete or incorrect information
df = df.drop(new_entries.index)


In [26]:
# Deleting movies with similar cast and title
not_docus = df[~df["genres"].isin(["Documentary"])]

not_docus_dup = not_docus[not_docus.duplicated(subset=["cast", "title"])]

df = df.drop(not_docus_dup.index)

In [ ]:
print("Number of rows:", df.shape[0])

515515

In [28]:
df.cast = df.cast.fillna('')
df.director = df.director.fillna('')
df.producers = df.producers.fillna('')
df.director_of_photography = df.director_of_photography.fillna('')
df.production_companies = df.production_companies.fillna('')
df.production_countries = df.production_countries.fillna('')
df.writers = df.writers.fillna('')
df.music_composer = df.music_composer.fillna('')
df.genres = df.genres.fillna('')